In [1]:
import numpy as np
import pandas as pd
import os

In [40]:
data_path = "/home/dimitribouche/Bureau/ENSAE/OnlineLearning/NYSE"
save_data_path = "/home/dimitribouche/Bureau/ENSAE/OnlineLearning"

In [12]:
def load_stocks(path):
    stock_data = dict()
    for stock in os.listdir(path):
        name = stock.split(".")[0]
        try:
            stock_data[name] = pd.read_table(path + "/" + stock, sep=",", index_col=0)["Close"]
            stock_data[name] .index = pd.to_datetime(stock_data[name] .index)
        except pd.errors.EmptyDataError:
            continue
    return stock_data   

In [13]:
nyse_stocks = load_stocks(data_path)

In [27]:
def date_index_union(stock_data):
    first = True
    for key in stock_data.keys():
        if first :
            date_union = stock_data[key].index
            first = False
        else :
            date_union = date_union.union(stock_data[key].index)
    return date_union
        

In [86]:
date_union = date_index_union(nyse_stocks)
most_recent = date_union[9500:]

In [87]:
def consolidate_data(stock_data, date_index):
    conso_data = pd.DataFrame(index=date_index)
    for stock in stock_data.keys():
        conso_data[stock] = stock_data[stock]
    return conso_data

In [88]:
# Consolidate the data according to the index most_recent (around 2005)
conso_data = consolidate_data(nyse_stocks, most_recent)

In [89]:
# Delete stocks which have "tolerance" or more Na value in the period
tolerance = 20
nrows = conso_data.shape[0]
cleaned = conso_data.dropna(axis=1, thresh=nrows-tolerance)
# Drop rows that have Na
cleaned = cleaned.dropna(axis=0, how="any")

In [90]:
cleaned

,nwl,fhn,cmi,cma,wfc,cms,nem,lmt,tgna,aig,...,tmo,brk-a,brk-b,pg,eog,hd,au,mck,hpq,tjx
Date,,,,,,,,,,,,,,,,,,,,,
1999-09-23,21.981,23.535,10.600,38.385,14.702,22.648,18.998,24.403,34.286,1115.27,...,12.389,60000.00,38.90,38.414,5.2386,37.638,25.9900,28.469,17.943,6.1404
1999-09-24,21.296,23.334,10.411,40.386,15.291,22.373,20.212,23.771,34.688,1096.81,...,11.694,59700.00,38.68,38.767,4.9135,37.709,27.9190,27.604,18.370,6.2000
1999-09-27,21.251,23.535,10.600,39.932,14.743,22.414,24.508,24.990,34.596,1082.09,...,11.850,58200.00,38.10,39.584,5.1094,37.357,33.1670,26.797,18.274,6.2818
1999-09-28,21.344,23.437,10.326,39.788,14.892,21.894,23.787,24.451,34.722,1072.01,...,11.636,56900.00,37.40,39.418,4.7849,37.493,33.2830,25.590,18.096,6.3230
1999-09-29,21.833,22.974,10.565,40.039,14.769,21.775,22.735,25.084,34.473,1056.73,...,11.636,56500.00,37.32,38.167,4.9469,37.991,32.3760,26.511,16.966,6.2683
1999-09-30,22.318,22.926,10.497,40.386,15.174,21.775,22.799,25.373,34.656,1072.95,...,11.636,55000.00,37.12,37.353,4.9906,38.914,30.3310,26.738,17.257,6.2545
1999-10-01,21.833,22.774,10.600,40.887,14.983,21.610,23.399,25.325,34.877,1048.90,...,11.743,55400.00,36.72,37.223,4.9906,39.445,30.8140,26.330,16.615,6.2271
1999-10-04,21.928,23.535,10.720,42.334,15.610,21.894,26.215,25.421,35.035,1058.24,...,12.074,57300.00,37.90,38.343,5.0189,40.366,32.3190,25.817,15.826,6.5055
1999-10-05,21.536,23.535,10.378,41.725,15.654,21.648,25.440,24.990,35.067,1082.82,...,11.801,57800.00,38.34,38.023,4.9330,40.084,31.7790,24.544,15.899,6.5190


In [91]:
# Save the data as a csv file
cleaned.to_csv(save_data_path + "/" + "DataNyse.csv")